In [5]:
import xarray as xr
import glob
import numpy as np
import pandas as pd

In [6]:
import sys
sys.path.append('/glade/u/home/linnia/ctsm6_ppe/')
from utils.pyfunctions import *

utils_path = '/glade/u/home/linnia/ctsm6_ppe/utils/'

2025-01-29 12:03:34.304814: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-29 12:03:35.546771: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-29 12:03:35.636228: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glade/u/apps/casper/23.10/spack/opt/spack/openmpi/4.1.6/oneapi/2023.2.1/dgcv/lib:/

In [90]:
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
ncores=1
nmem='10GB'
cluster = PBSCluster(
    cores=ncores, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus='+str(ncores)+':mem='+nmem, # Specify resources
    project='P93300041', # Input your project ID here
    walltime='01:00:00', # Amount of wall time
    #interface='ib0', # Interface to use
)

# Scale up
cluster.scale(10)

# Setup your client
client = Client(cluster)

In [92]:
yy = (1985,2023)
dir='/glade/campaign/cgd/tss/projects/PPE/ctsm6_oaat/hist/'
key = '/glade/campaign/cgd/tss/projects/PPE/ctsm6_oaat/ctsm6_oaat_key.csv'
exp='oaat'
tape='h0'

dvs=['GPP','AR','HR','NPP','NBP','NEP','ER',
     'EFLX_LH_TOT','FCTR','FCEV','FGEV','BTRANMN','FGR','FSH',
     'SOILWATER_10CM','TWS','QRUNOFF','SNOWDP','H2OSNO','FSNO',
     'TLAI','FSR','ALTMAX','TV','TG','NPP_NUPTAKE'
     'FAREA_BURNED','COL_FIRE_CLOSS',
     'TOTVEGC','TOTECOSYSC','TOTSOMC_1m',
     'TOTVEGN','TOTECOSYSN']

#dvs = ['VEGWP','TOTVEGC','NPP','NPP_NUPTAKE','HTOP','HR','GPP','TLAI','FCTR','FGEV','FCEV','BTRANMN','AR','AGNPP']


In [93]:
ds= get_exp(exp,dir,key,dvs,tape,yy,utils_path)

In [91]:
# calculate global and biome mean, std, amp
la=xr.open_dataset(utils_path+'landarea_retrain_h0.nc').landarea
b=xr.open_dataset(utils_path+'whit/whitkey_CRUJRA.nc').biome
out=xr.Dataset()
    
for v in dvs:

        x=amean(ds[v])
        amp=(ds[v].groupby('time.year').max()-ds[v].groupby('time.year').min()).mean(dim='year').compute()

        out[v+'_gridded_mean']=x.mean(dim='year')

        out[v+'_global_amp'] =gmean(amp,la)
        out[v+'_global_std'] =gmean(x.std(dim='year'),la)
        out[v+'_global_mean']=gmean(x.mean(dim='year'),la)

        out[v+'_biome_amp']  =bmean(amp,la,b)
        out[v+'_biome_std']  =bmean(x.std(dim='year'),la,b)
        out[v+'_biome_mean'] =bmean(x.mean(dim='year'),la,b)

        for dv in out.data_vars:
            if v in dv:
                out[dv].attrs=ds[v].attrs

KeyboardInterrupt: 

In [7]:
df_key = pd.read_csv(key)

out = out.assign_coords(
    key=('ens', df_key['key'].values),
    param=('ens', df_key['param'].values),
    minmax=('ens', df_key['minmax'].values)
)

out = out.set_index(ens=["param", "minmax", "key"], append=True)
ds_out = out.reset_index("ens")

In [12]:
ds_out.to_netcdf('/glade/campaign/cgd/tss/projects/PPE/ctsm6_oaat/postp/ctsm6_oaat_postp_1985-2023.nc')

### test postprocessing PFT

In [103]:
lhc='lhc0000'

In [148]:
# setup 
dir='/glade/campaign/cgd/tss/projects/PPE/ctsm6_lhc/hist/'
out_dir='/glade/work/linnia/CLM6-PPE/ctsm6_lhc/postp/tmp/'
tape='h1'

dvs = ['TOTVEGC','NPP','NPP_NUPTAKE','HTOP','GPP','TLAI','FCTR','FGEV','FCEV','BTRANMN','AR','AGNPP','pfts1d_itype_veg']

def pp(ds):
    yr0=str(ds['time.year'][0].values)
    nt=len(ds.time)
    ds['time'] = xr.cftime_range(yr0,periods=nt,freq='MS',calendar='noleap') #fix time bug
    return ds[dvs]

In [153]:
f=sorted(glob.glob(dir+'*'+lhc+'*.'+tape+'.*'))
ds=xr.open_mfdataset(f,combine='by_coords',preprocess=pp)

# calculate pft mean
yr1 = '1985'
yr2 = '2023'
pft=ds.pfts1d_itype_veg.isel(time=0)
la=xr.open_dataset(utils_path+'lapft_ctsm5.3.012.nc')
la_pft = la.lapfty.sel(year=slice(yr1,yr2))
la_pft = la_pft.assign_coords(pft=pft)

out=xr.Dataset()
    
for v in dvs:

        x=amean(ds[v].sel(time=slice(yr1,yr2)))
        X = x.assign_coords(pft=pft)
        out[v+'_pft_mean'] = pmean(X,la_pft)

        for dv in out.data_vars:
            if v in dv:
                out[dv].attrs=ds[v].attrs
